In [1]:
import os
from PIL import Image, UnidentifiedImageError
from PIL.TiffTags import TAGS
import pandas as pd
import numpy as np
import cv2

import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe" #il faut mettre là où tesseract est installé

import time
from tqdm.notebook import tqdm

In [2]:
#################################### A MODIFIER SELON SA PROPRE ARBORESCENCE
project_path = 'D:/Projet/mai25_bds_extraction/' 
####################################

raw_data_path = os.path.join(project_path, 'data', 'raw')
processed_data_path = os.path.join(project_path, 'data', 'processed')

rvl_cdip_images_path = os.path.join(raw_data_path, 'RVL-CDIP', 'images')

In [3]:
def get_rvl_cdip_image_files(path):
    tmp_list = []
    for foldername, _, filenames in os.walk(path):
        if filenames:
            filename = filenames[0]
            # we check that the structure is relevant to our expectation with 2 assert
            assert len(filenames) == 1
            if filename.startswith('.'): # avoid to consider files like .DS_Store on mac
                continue
            assert filename.endswith(".tif"), f"{foldername},{filename}"
            tmp_list.append([
                os.path.relpath(foldername, rvl_cdip_images_path),
                filename
            ])
    return pd.DataFrame(tmp_list, columns = ["relative_path", "filename"])


In [4]:
t = time.time()
df = get_rvl_cdip_image_files(rvl_cdip_images_path)
print(f"Duree d'exécution: {time.time() - t:.3f} secondes.")
print(f"({len(df)} images traitées)\n")

df.head()

Duree d'exécution: 102.193 secondes.
(400000 images traitées)



,relative_path,filename
0,imagesa\a\a\a\aaa06d00,50486482-6482.tif
1,imagesa\a\a\a\aaa08d00,2072197187.tif
2,imagesa\a\a\a\aaa09e00,2029372116.tif
3,imagesa\a\a\a\aaa10c00,2085133627a.tif
4,imagesa\a\a\a\aaa11d00,515558347+-8348.tif


## Que veut-on extraire des images ? 
Dans un premier notebook on avait extrait :
- format (en fait tout est en .tiff)
- taille (longueur et largeur, normalement, la plus grande dimension de l'image devrait être mise à 1000)
- mode (ici en fait ce sont toutes des images en nuances de gris, sur 8 bits)
- nombre de pages par image
- niveau de flou - grace à la variance du laplacien
- le bruit de l'image (quand des points apparaîssent sur un fond censé être blanc par exemple)
- densité (quantité de pixels noirs par rapport à la quantité de pixels blancs)

Lors de cette 2e itération, sont extraits:
- le nombre de blocs de texte
- la quantité de blanc autour du contenu/présence d'une bordure
- la présence de motifs réguliers
- la détection des caractères de texte

In [8]:
# on cherche le nombre de blocs de texte
""" Méthode :
Binarisation avec seuillage adaptatif
Morphologie pour connecter les lettres (dilatation légère)
cv2.findContours pour détecter les blocs
Filtrage des très petits éléments (bruit)
"""
def blocs(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    blurred = cv2.medianBlur(img, 3)
    thresh = cv2.adaptiveThreshold(blurred, 255, 
                                   cv2.ADAPTIVE_THRESH_MEAN_C, 
                                   cv2.THRESH_BINARY_INV, 15, 10)
    
    # Dilatation horizontale forte pour relier les mots/lignes
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 3))
    dilated = cv2.dilate(thresh, kernel, iterations=1)

    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    blocks = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        area = w * h
        if area > 5000 and h > 10:  # Élimine petits bruits et points isolés
            blocks.append((x, y, w, h))

    return len(blocks)

In [9]:
#on cherche la taille de la bordure blanche si elle existe
""" Méthode :
Binarisation inverse
Bounding box du contenu
Mesure des marges gauche/droite/haut/bas
"""
def bordure(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    coords = cv2.findNonZero(binary)
    x, y, w, h = cv2.boundingRect(coords)
    h_img, w_img = img.shape

    top_margin = y / h_img
    bottom_margin = (h_img - (y + h)) / h_img
    left_margin = x / w_img
    right_margin = (w_img - (x + w)) / w_img

    return {
        'top_margin': top_margin,
        'bottom_margin': bottom_margin,
        'left_margin': left_margin,
        'right_margin': right_margin
    }

In [10]:
# on cherche des motids dans la dimension de fourrier
""" Méthode :
Transformer l’image avec FFT
Observer la densité hors du centre
Ratio d’énergie dans les hautes fréquences
"""
def motifs(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    magnitude = np.abs(fshift)

    h, w = magnitude.shape
    center = magnitude[h//2-10:h//2+10, w//2-10:w//2+10]
    high_freq = magnitude.sum() - center.sum()
    ratio = high_freq / magnitude.sum()
    
    return ratio

In [11]:
# détection des caracteres avec tesseract

import pytesseract

def features(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    text = pytesseract.image_to_string(img)
    
    nb_lines = text.count('\n')
    nb_words = len(text.split())
    nb_digits = sum(c.isdigit() for c in text)
    nb_letters = sum(c.isalpha() for c in text)

    return {
        'num_lines': nb_lines,
        'num_words': nb_words,
        'digit_ratio': nb_digits / (nb_digits + nb_letters + 1e-5),
    }

# Attention
### Dans la cellule suivante, 5 coeurs de l'ordinateur sont utilisés (n_jobs = 5), à changer si besoin

In [12]:
from joblib import Parallel, delayed
import pandas as pd
import os
import time

def get_image_data(file_path):
    if not os.path.exists(file_path):
        print(f"[Manquant] {file_path}")
        return [np.nan] * 4
    try:
        bloc = blocs(file_path)
        feature = features(file_path)
        bord = bordure(file_path)
        motif = motifs(file_path)
        return bloc, feature, bord, motif
    except Exception as e:
        print(f"[Erreur lecture] {file_path} → {e}")
        return [np.nan] * 4

def process_one_image(row_dict, base_path):
    relative_path = row_dict["relative_path"]
    filename = row_dict["filename"]
    file_path = os.path.join(base_path, relative_path, filename)
    return get_image_data(file_path)

#maintenant on travaille en parallèle sur 5 coeurs (j'en ai 8 sur mon ordinateur)
base_path = rvl_cdip_images_path
n_jobs = 5

t = time.time()
results = Parallel(n_jobs=n_jobs, backend="threading")(
    delayed(process_one_image)(row.to_dict(), base_path) for _, row in df.iterrows()
)
print(f"Durée d'exécution: {time.time() - t:.3f} secondes.")
print(f"({len(results)} images traitées)")


[Erreur lecture] D:/Projet/mai25_bds_extraction/data\raw\RVL-CDIP\images\imagese\e\j\e\eje42e00\2500126531_2500126536.tif → OpenCV(4.10.0) C:\b\abs_a3k2mim6t7\croot\opencv-suite_1744821726920\work\modules\imgproc\src\median_blur.dispatch.cpp:283: error: (-215:Assertion failed) !_src0.empty() in function 'cv::medianBlur'

Durée d'exécution: 203056.436 secondes.
(400000 images traitées)


In [13]:
results_df = pd.DataFrame(results, columns=["bloc", "feature", "bord", "motif"], index=df.index)
df = pd.concat([df, results_df], axis=1)
df.to_csv(os.path.join(project_path, 'data', 'extracted', 'rvl_cdip_draft_2.csv'))